In [1]:
# Install all required packages
!pip install flask flask-cors openai python-dotenv sqlalchemy flask-sqlalchemy requests
!pip install torch transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import json
import sqlite3
from datetime import datetime
import requests
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
import os
from dotenv import load_dotenv
import uuid

# Load environment variables
load_dotenv()

# Initialize Flask app
app = Flask(__name__)
CORS(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///customer_support.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

In [3]:
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy(app)

class ConversationSession(db.Model):
    id = db.Column(db.String(36), primary_key=True)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    updated_at = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

class Message(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    session_id = db.Column(db.String(36), db.ForeignKey('conversation_session.id'))
    role = db.Column(db.String(20))  # 'user' or 'assistant'
    content = db.Column(db.Text)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

# Create tables
with app.app_context():
    db.create_all()

In [4]:
# Sample FAQ dataset
faq_dataset = {
    "shipping": {
        "question": "What are your shipping options?",
        "answer": "We offer standard shipping (3-5 business days) for $5.99, express shipping (1-2 business days) for $12.99, and free shipping on orders over $50."
    },
    "returns": {
        "question": "What is your return policy?",
        "answer": "We accept returns within 30 days of purchase. Items must be in original condition with tags attached. Refunds will be processed within 5-7 business days."
    },
    "payment": {
        "question": "What payment methods do you accept?",
        "answer": "We accept Visa, MasterCard, American Express, PayPal, and Apple Pay."
    },
    "tracking": {
        "question": "How can I track my order?",
        "answer": "You can track your order using the tracking number sent to your email. Alternatively, login to your account and check order history."
    },
    "warranty": {
        "question": "Do your products come with warranty?",
        "answer": "Yes, all our products come with a 1-year manufacturer warranty. Contact support with your purchase details for warranty claims."
    }
}

# Save FAQ dataset to file
with open('faq_dataset.json', 'w') as f:
    json.dump(faq_dataset, f, indent=2)

In [5]:
class CustomerSupportLLM:
    def __init__(self):
        self.faq_dataset = faq_dataset
        self.escalation_threshold = 3  # Number of failed attempts before escalation
        
    def find_best_faq_match(self, query):
        """Find the best matching FAQ for a user query"""
        query_lower = query.lower()
        best_match = None
        best_score = 0
        
        for key, faq in self.faq_dataset.items():
            score = 0
            faq_text = f"{faq['question']} {faq['answer']}".lower()
            
            # Simple keyword matching (can be enhanced with embeddings)
            keywords = ['shipping', 'return', 'payment', 'track', 'warranty', 
                       'delivery', 'refund', 'money', 'card', 'pay']
            
            for keyword in keywords:
                if keyword in query_lower and keyword in faq_text:
                    score += 1
            
            if score > best_score:
                best_score = score
                best_match = key
        
        return best_match, best_score
    
    def generate_response(self, user_query, conversation_history, failed_attempts=0):
        """Generate response using FAQ matching and simple rule-based LLM simulation"""
        
        # Check if we need to escalate
        if failed_attempts >= self.escalation_threshold:
            return {
                "response": "I'm having trouble helping you with this issue. Let me connect you with a human agent for better assistance.",
                "escalated": True,
                "confidence": 0.0,
                "faq_match": None
            }
        
        # Find best FAQ match
        best_match, confidence = self.find_best_faq_match(user_query)
        
        if best_match and confidence > 0:
            faq = self.faq_dataset[best_match]
            return {
                "response": faq['answer'],
                "escalated": False,
                "confidence": confidence / 5.0,  # Normalize confidence
                "faq_match": best_match
            }
        else:
            # Generic response for unmatched queries
            alternative_responses = [
                "I understand you're asking about that. Could you provide more details so I can assist you better?",
                "I want to make sure I help you correctly. Could you rephrase your question?",
                "I'm not sure I understand completely. Can you give me more context about your issue?"
            ]
            
            return {
                "response": alternative_responses[failed_attempts % len(alternative_responses)],
                "escalated": False,
                "confidence": 0.0,
                "faq_match": None
            }
    
    def summarize_conversation(self, conversation_history):
        """Summarize the conversation for human agents"""
        summary = "Conversation Summary:\n"
        for msg in conversation_history:
            summary += f"{msg['role']}: {msg['content']}\n"
        return summary
    
    def suggest_next_actions(self, conversation_history):
        """Suggest next actions based on conversation"""
        last_user_message = None
        for msg in reversed(conversation_history):
            if msg['role'] == 'user':
                last_user_message = msg['content'].lower()
                break
        
        if last_user_message:
            if any(word in last_user_message for word in ['shipping', 'delivery', 'track']):
                return "Consider asking for order number to track shipping"
            elif any(word in last_user_message for word in ['return', 'refund', 'exchange']):
                return "Request order details and reason for return"
            elif any(word in last_user_message for word in ['payment', 'card', 'pay']):
                return "Verify payment method and transaction details"
        
        return "Ask for more details about the customer's issue"

# Initialize LLM
support_llm = CustomerSupportLLM()

In [6]:
@app.route('/')
def home():
    return jsonify({"message": "AI Customer Support Bot API is running!"})

@app.route('/api/start_session', methods=['POST'])
def start_session():
    """Start a new conversation session"""
    session_id = str(uuid.uuid4())
    new_session = ConversationSession(id=session_id)
    db.session.add(new_session)
    db.session.commit()
    
    return jsonify({
        "session_id": session_id,
        "message": "New session started"
    })

@app.route('/api/chat', methods=['POST'])
def chat():
    """Handle customer queries"""
    data = request.json
    session_id = data.get('session_id')
    user_message = data.get('message')
    
    if not session_id or not user_message:
        return jsonify({"error": "session_id and message are required"}), 400
    
    # Verify session exists
    session = ConversationSession.query.get(session_id)
    if not session:
        return jsonify({"error": "Invalid session_id"}), 404
    
    # Get conversation history
    history_messages = Message.query.filter_by(session_id=session_id).order_by(Message.timestamp).all()
    conversation_history = [
        {"role": msg.role, "content": msg.content} 
        for msg in history_messages
    ]
    
    # Count failed attempts (messages without FAQ matches)
    failed_attempts = sum(1 for msg in history_messages if msg.role == 'user' and not any(
        faq_keyword in msg.content.lower() for faq_keyword in ['shipping', 'return', 'payment', 'track', 'warranty']
    ))
    
    # Save user message
    user_msg = Message(session_id=session_id, role='user', content=user_message)
    db.session.add(user_msg)
    
    # Generate response
    llm_response = support_llm.generate_response(
        user_message, 
        conversation_history, 
        failed_attempts
    )
    
    # Save assistant response
    assistant_msg = Message(
        session_id=session_id, 
        role='assistant', 
        content=llm_response['response']
    )
    db.session.add(assistant_msg)
    db.session.commit()
    
    # Update session timestamp
    session.updated_at = datetime.utcnow()
    db.session.commit()
    
    return jsonify({
        "response": llm_response['response'],
        "escalated": llm_response['escalated'],
        "confidence": llm_response['confidence'],
        "faq_match": llm_response['faq_match'],
        "session_id": session_id
    })

@app.route('/api/conversation/<session_id>', methods=['GET'])
def get_conversation(session_id):
    """Get conversation history for a session"""
    messages = Message.query.filter_by(session_id=session_id).order_by(Message.timestamp).all()
    
    conversation = [
        {
            "role": msg.role,
            "content": msg.content,
            "timestamp": msg.timestamp.isoformat()
        }
        for msg in messages
    ]
    
    return jsonify({
        "session_id": session_id,
        "conversation": conversation
    })

@app.route('/api/summarize/<session_id>', methods=['GET'])
def summarize_conversation(session_id):
    """Get conversation summary"""
    messages = Message.query.filter_by(session_id=session_id).order_by(Message.timestamp).all()
    
    conversation_history = [
        {"role": msg.role, "content": msg.content} 
        for msg in messages
    ]
    
    summary = support_llm.summarize_conversation(conversation_history)
    suggestions = support_llm.suggest_next_actions(conversation_history)
    
    return jsonify({
        "summary": summary,
        "suggestions": suggestions
    })

In [7]:
HTML_INTERFACE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Customer Support Bot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .chat-container {
            background: white;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            overflow: hidden;
        }
        .chat-header {
            background: #007bff;
            color: white;
            padding: 20px;
            text-align: center;
        }
        .chat-messages {
            height: 400px;
            overflow-y: auto;
            padding: 20px;
            border-bottom: 1px solid #eee;
        }
        .message {
            margin-bottom: 15px;
            padding: 10px 15px;
            border-radius: 20px;
            max-width: 70%;
        }
        .user-message {
            background: #007bff;
            color: white;
            margin-left: auto;
        }
        .assistant-message {
            background: #f1f1f1;
            color: #333;
        }
        .escalated-message {
            background: #ff6b6b;
            color: white;
        }
        .chat-input {
            padding: 20px;
            display: flex;
            gap: 10px;
        }
        .chat-input input {
            flex: 1;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
        .chat-input button {
            padding: 10px 20px;
            background: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        .session-info {
            background: #e9ecef;
            padding: 10px;
            border-radius: 5px;
            margin-bottom: 10px;
            font-size: 12px;
        }
        .confidence-badge {
            font-size: 10px;
            background: #28a745;
            color: white;
            padding: 2px 6px;
            border-radius: 10px;
            margin-left: 10px;
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="chat-header">
            <h2>🤖 AI Customer Support</h2>
            <p>Ask me about shipping, returns, payments, and more!</p>
        </div>
        
        <div class="session-info">
            Session ID: <span id="sessionId">Not started</span>
            <button onclick="startNewSession()" style="margin-left: 10px; padding: 5px 10px; font-size: 10px;">New Session</button>
        </div>
        
        <div class="chat-messages" id="chatMessages">
            <div class="message assistant-message">
                Hello! I'm your AI customer support assistant. How can I help you today?
            </div>
        </div>
        
        <div class="chat-input">
            <input type="text" id="userInput" placeholder="Type your question here..." onkeypress="handleKeyPress(event)">
            <button onclick="sendMessage()">Send</button>
        </div>
    </div>

    <script>
        let currentSessionId = null;
        
        async function startNewSession() {
            try {
                const response = await fetch('/api/start_session', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    }
                });
                const data = await response.json();
                currentSessionId = data.session_id;
                document.getElementById('sessionId').textContent = currentSessionId;
                document.getElementById('chatMessages').innerHTML = 
                    '<div class="message assistant-message">Hello! I\'m your AI customer support assistant. How can I help you today?</div>';
            } catch (error) {
                console.error('Error starting session:', error);
            }
        }
        
        async function sendMessage() {
            const userInput = document.getElementById('userInput');
            const message = userInput.value.trim();
            
            if (!message) return;
            
            if (!currentSessionId) {
                await startNewSession();
            }
            
            // Add user message to chat
            addMessageToChat('user', message);
            userInput.value = '';
            
            try {
                const response = await fetch('/api/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({
                        session_id: currentSessionId,
                        message: message
                    })
                });
                
                const data = await response.json();
                
                let messageClass = 'assistant-message';
                if (data.escalated) {
                    messageClass = 'escalated-message';
                }
                
                let confidenceBadge = '';
                if (data.confidence > 0) {
                    confidenceBadge = `<span class="confidence-badge">${(data.confidence * 100).toFixed(0)}% match</span>`;
                }
                
                addMessageToChat('assistant', data.response + confidenceBadge, messageClass);
                
            } catch (error) {
                console.error('Error sending message:', error);
                addMessageToChat('assistant', 'Sorry, I encountered an error. Please try again.', 'assistant-message');
            }
        }
        
        function addMessageToChat(role, content, messageClass = null) {
            const chatMessages = document.getElementById('chatMessages');
            const messageDiv = document.createElement('div');
            
            if (!messageClass) {
                messageClass = role === 'user' ? 'user-message' : 'assistant-message';
            }
            
            messageDiv.className = `message ${messageClass}`;
            messageDiv.innerHTML = content;
            
            chatMessages.appendChild(messageDiv);
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }
        
        function handleKeyPress(event) {
            if (event.key === 'Enter') {
                sendMessage();
            }
        }
        
        // Start a session when page loads
        window.onload = startNewSession;
    </script>
</body>
</html>
"""

@app.route('/chat-ui')
def chat_ui():
    return render_template_string(HTML_INTERFACE)

In [ ]:
# === REPLACE YOUR EXISTING "RUN APPLICATION" CELL WITH THIS ===
# Cell 9: Updated Main Application Runner

if __name__ == '__main__':
    # Find a free port
    try:
        FINAL_PORT = find_free_port(5000)
        print(f"🎯 Using port: {FINAL_PORT}")
    except:
        FINAL_PORT = 5001
        print(f"⚠️ Using fallback port: {FINAL_PORT}")
    
    print("🚀 Starting AI Customer Support Bot...")
    print(f"📱 Access the chat interface at: http://localhost:{FINAL_PORT}/chat-ui")
    print("🔗 API endpoints:")
    print("  POST /api/start_session - Start new conversation")
    print("  POST /api/chat - Send message")
    print("  GET  /api/conversation/<session_id> - Get conversation history")
    print("  GET  /api/summarize/<session_id> - Get conversation summary")
    print("\n💡 Try these example queries:")
    print("  - What are your shipping options?")
    print("  - How can I return a product?")
    print("  - Do you accept PayPal?")
    print("  - I want to track my order")
    
    try:
        app.run(host='0.0.0.0', port=FINAL_PORT, debug=False, use_reloader=False)
    except Exception as e:
        print(f"❌ Error: {e}")
        print("🔄 Trying alternative port 5001...")
        app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)

⚠️ Using fallback port: 5001
🚀 Starting AI Customer Support Bot...
📱 Access the chat interface at: http://localhost:5001/chat-ui
🔗 API endpoints:
  POST /api/start_session - Start new conversation
  POST /api/chat - Send message
  GET  /api/conversation/<session_id> - Get conversation history
  GET  /api/summarize/<session_id> - Get conversation summary

💡 Try these example queries:
  - What are your shipping options?
  - How can I return a product?
  - Do you accept PayPal?
  - I want to track my order
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.25.232.54:5001
Press CTRL+C to quit


In [ ]:
# === ADD THIS AS A NEW CELL AFTER YOUR HTML INTERFACE ===
# Cell 8: Port Conflict Solution

import os
import socket
import subprocess
import sys
import time

def find_free_port(start_port=5000, max_attempts=100):
    """Find a free port starting from start_port"""
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('localhost', port))
                return port
        except OSError:
            continue
    raise Exception(f"No free ports found in range {start_port}-{start_port + max_attempts}")

def stop_port(port):
    """Stop processes using specified port"""
    try:
        if sys.platform == "win32":
            subprocess.run(f'netstat -ano | findstr :{port}', shell=True)
        else:
            result = subprocess.run(['lsof', '-ti', f':{port}'], capture_output=True, text=True)
            if result.stdout.strip():
                pids = result.stdout.strip().split('\n')
                for pid in pids:
                    if pid:
                        subprocess.run(['kill', '-9', pid], capture_output=True)
        time.sleep(1)
    except:
        pass

# Stop common ports that might conflict
for port in [5000, 5001, 5002, 5003]:
    stop_port(port)

print("✅ Port cleanup completed")